In [ ]:
!pip install nibabel

In [ ]:
import numpy
import warnings
from keras.layers import  Conv3D, Conv3DTranspose, Input, Concatenate, Add,  RepeatVector, Activation #Convolution3D,
from keras import activations
from keras.models import Model
from keras.layers.advanced_activations import PReLU
from keras import activations, initializers, regularizers #initializations -> initializers
from keras.engine import Layer, InputSpec
from keras.utils.conv_utils import conv_output_length #np_utils
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from keras.engine.topology import Layer
import functools
import tensorflow as tf
import nibabel as nib
import pickle
import time
import glob
from os import listdir
#import SimpleITK
import os.path
from data_augmentation_nii import DataGenerator

## Construcción del modelo
-----------------

In [ ]:
def downward_layer(input_layer, n_convolutions, n_output_channels):
    inl = input_layer
    for _ in range(n_convolutions-1):
        inl = PReLU()(
            #Convolution3D(n_output_channels // 2, 5, 5, 5, border_mode='same', dim_ordering='tf')(inl)
            Conv3D(n_output_channels // 2, (5, 5, 5), padding='same', data_format="channels_last")(inl)
        )
    conv = Conv3D(n_output_channels // 2, (5, 5, 5), padding='same', data_format='channels_last')(inl)
    #add = merge([conv, input_layer], mode='sum')
    add = Add()([conv, input_layer])
    downsample = Conv3D(n_output_channels, (2,2,2), strides=(2,2,2))(add)
    prelu = PReLU()(downsample)
    return prelu, add

In [ ]:
def upward_layer(input0 ,input1, n_convolutions, n_output_channels):
    #merged = merge([input0, input1], mode='concat', concat_axis=4)
    merged = Concatenate()([input0, input1]) #concat_axis = 4 -> axis = -1, último eje, los canales o feature maps
    inl = merged
    for _ in range(n_convolutions-1):
        inl = PReLU()(
            Conv3D(n_output_channels * 4, (5, 5, 5), padding='same', data_format= "channels_last")(inl)
        )
    conv = Conv3D(n_output_channels * 4, (5, 5, 5), padding='same', data_format= "channels_last")(inl)
    add = Add()([conv, merged])
    shape = add.get_shape().as_list()
    new_shape = (1, shape[1] * 2, shape[2] * 2, shape[3] * 2, n_output_channels)
    upsample = Conv3DTranspose(n_output_channels, (4,4,4), strides=(2,2,2), padding = 'same')(add)
    return PReLU()(upsample)

In [ ]:
# Layer 1
#input_layer = Input(shape=(128, 128, 64, 1), name='data')
input_layer = Input(shape=(40, 40, 40, 1), name='data')
conv_1 = Conv3D(16, (5, 5, 5), padding= "same", data_format= "channels_last")(input_layer)
repeat_1 = Concatenate()([input_layer] * 16)
add_1 = Add()([conv_1, repeat_1])
prelu_1_1 = PReLU()(add_1)
downsample_1 = Conv3D(32, (2,2,2), strides=(2,2,2))(prelu_1_1)
prelu_1_2 = PReLU()(downsample_1)

# Layer 2,3,4
out2, left2 = downward_layer(prelu_1_2, 2, 64)
out3, left3 = downward_layer(out2, 3, 128)
#out4, left4 = downward_layer(out3, 3, 256)

# Layer 5
#conv_5_1 = Conv3D(256, (5, 5, 4), padding='same', data_format="channels_last")(out4)
conv_5_1 = Conv3D(128, (5, 5, 4), padding='same', data_format="channels_last")(out3)
prelu_5_1 = PReLU()(conv_5_1)
#conv_5_2 = Conv3D(256, (5, 5, 4), padding='same', data_format= "channels_last")(prelu_5_1)
conv_5_2 = Conv3D(128, (5, 5, 4), padding='same', data_format= "channels_last")(prelu_5_1)
prelu_5_2 = PReLU()(conv_5_2)
#conv_5_3 = Conv3D(256, (5, 5, 4), padding='same', data_format= "channels_last")(prelu_5_2)
conv_5_3 = Conv3D(128, (5, 5, 4), padding='same', data_format= "channels_last")(prelu_5_2)
#add_5 = Add()([conv_5_3, out4])
add_5 = Add()([conv_5_3, out3])
prelu_5_1 = PReLU()(add_5)
#downsample_5 = Conv3DTranspose(128, (2,2,2), strides=(2,2,2), padding = 'same')(prelu_5_1)
downsample_5 = Conv3DTranspose(64, (2,2,2), strides=(2,2,2), padding = 'same')(prelu_5_1)
prelu_5_2 = PReLU()(downsample_5)

#Layer 6,7,8
#out6 = upward_layer(prelu_5_2, left4, 3, 64)
#out7 = upward_layer(out6, left3, 3, 32)
out7 = upward_layer(prelu_5_2, left3, 3, 32)
out8 = upward_layer(out7, left2, 2, 16)

#Layer 9
merged_9 = Concatenate()([out8, add_1])
conv_9_1 = Conv3D(32, (5, 5, 5), padding='same', data_format= "channels_last")(merged_9)
add_9 = Add()([conv_9_1, merged_9])
output = Conv3D(1, (1, 1, 1), padding='same', data_format= "channels_last", activation='sigmoid')(add_9)

model = Model(input_layer, output)

In [ ]:
model.summary(line_length=113)

In [ ]:
plot_model(model, to_file="model.png")

### Función de pérdida Dice Coefficient
----------------------

In [ ]:
def dice_coe(output, target, loss_type='sorensen', axis=(1, 2, 3), weighted=False, smooth=1e-5):
    """Soft dice (Sørensen or Jaccard) coefficient for comparing the similarity
    of two batch of data, usually be used for binary image segmentation
    i.e. labels are binary. The coefficient between 0 to 1, 1 means totally match.
    Parameters
    -----------
    output : Tensor
      A distribution with shape: [batch_size, ....], (any dimensions).
    target : Tensor
      The target distribution, format the same with `output`.
    loss_type : str
      ``jaccard`` or ``sorensen``, default is ``jaccard``.
    axis : tuple of int
      All dimensions are reduced, default ``[1,2,3]``.
    weight : bool
      Boolean option for generalized dice loss
    smooth : float
      This small value will be added to the numerator and denominator.
        - If both output and target are empty, it makes sure dice is 1.
        - If either output or target are empty (all pixels are background), dice = ```smooth/(small_value + smooth)``, then if smooth is very small, dice close to 0 (even the image values lower than the threshold), so in this case, higher smooth can have a higher dice.
    Examples
    ---------
    >>> import tensorlayer as tl
    >>> outputs = tl.act.pixel_wise_softmax(outputs)
    >>> dice_loss = 1 - tl.cost.dice_coe(outputs, y_)
    References
    -----------
    - `Wiki-Dice <https://en.wikipedia.org/wiki/Sørensen–Dice_coefficient>`__
    """
    output_ = output[:,:,:,:,0]
    target_ = target[:,:,:,:,0]
    inse = tf.reduce_sum(output_ * target_, axis=axis)
    if loss_type == 'jaccard':
      l = tf.reduce_sum(output_ * output_, axis=axis)
      r = tf.reduce_sum(target_ * target_, axis=axis)
    elif loss_type == 'sorensen':
      l = tf.reduce_sum(output_, axis=axis)
      r = tf.reduce_sum(target_, axis=axis)
    else:
      raise Exception("Unknown loss_type")

    if weighted:
      w = 1/(tf.reduce_sum(target_*target_, axis=axis) + smooth)
      dice = tf.reduce_sum(2.* w * inse +smooth, axis=-1)/tf.reduce_sum(w*(l + r + smooth),axis=-1)
      dice = tf.reduce_mean(dice, name='dice_coe')
    else:
      # old axis=[0,1,2,3]
      # dice = 2 * (inse) / (l + r)
      # epsilon = 1e-5
      # dice = tf.clip_by_value(dice, 0, 1.0-epsilon) # if all empty, dice = 1
      # new haodong
      dice = (2. * inse + smooth) / (l + r + smooth)
      dice = tf.reduce_mean(dice, name='dice_coe')

    return dice

In [ ]:
def dice_coe_loss(y_true, y_pred):
    return 1 - dice_coe(y_pred, y_true)

### Data Generator

In [ ]:
train_dir = './drive/MyDrive/VNet/data/train' 
val_dir = './drive/MyDrive/VNet/data/val'
file_paths_train = glob.glob(os.path.join(train_dir, 't2/*.nii.gz'))
file_paths_val = glob.glob(os.path.join(val_dir, 't2/*.nii.gz'))
ids_train = [path_t[-13:] for path_t in file_paths_train]
ids_val = [path_v[-13:] for path_v in file_paths_val]

train_datagen = DataGenerator(ids_train, train_dir,
                              n_samples = len(ids_train),
                              batch_size=2, 
                              image_shape=(40,40,40), 
                              rotation_range=0.2,
                              shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              flip=True)

val_datagen = DataGenerator(ids_val, val_dir,
                              n_samples = len(ids_val),
                              batch_size=2, 
                              image_shape=(40,40,40), 
                              augmentation=False)

### Compilación y ejecución

In [ ]:
model.compile(optimizer=Adam(lr=1e-5), loss=dice_coe_loss, metrics=[dice_coe])
#model.compile(optimizer=Adam(lr=1e-5), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss', save_best_only=True)

history = model.fit(train_datagen,
                    steps_per_epoch=train_datagen.n_samples/train_datagen.batch_size,
                    epochs=3000,
                    verbose=1, 
                    validation_data=val_datagen, 
                    validation_steps=val_datagen.n_samples/val_datagen.batch_size)

Se han truncado las últimas 5000 líneas del flujo de salida.
Epoch 501/3000
27/27 [==============================] - 4s 135ms/step - loss: 0.2001 - dice_coe: 0.7999 - val_loss: 0.2902 - val_dice_coe: 0.7098
Epoch 502/3000
27/27 [==============================] - 4s 135ms/step - loss: 0.1832 - dice_coe: 0.8168 - val_loss: 0.3386 - val_dice_coe: 0.6614
Epoch 503/3000
27/27 [==============================] - 4s 136ms/step - loss: 0.1719 - dice_coe: 0.8281 - val_loss: 0.2567 - val_dice_coe: 0.7433
Epoch 504/3000
27/27 [==============================] - 4s 136ms/step - loss: 0.1849 - dice_coe: 0.8151 - val_loss: 0.2734 - val_dice_coe: 0.7266
Epoch 505/3000
27/27 [==============================] - 4s 137ms/step - loss: 0.1601 - dice_coe: 0.8399 - val_loss: 0.3220 - val_dice_coe: 0.6780
Epoch 506/3000
27/27 [==============================] - 4s 138ms/step - loss: 0.2200 - dice_coe: 0.7800 - val_loss: 0.2651 - val_dice_coe: 0.7349
Epoch 507/3000
27/27 [==============================] - 4s 137m

In [ ]:
test_images = glob.glob('./drive/MyDrive/VNet/data/val/t2/*.nii.gz')

for test_image in test_images:
  img = nib.load(test_image)
  img_data = img.get_fdata()
  img_data = img_data.reshape((1,) + img_data.shape + (1,))
  prediction = model.predict(img_data)
  predict_nii = nib.Nifti1Image(prediction[0][:,:,:,0], img.affine, img.header)
  nib.save(predict_nii, f'./drive/MyDrive/VNet/data/val/{test_image[-13:]}')

In [ ]:
img_test = nib.load('./drive/MyDrive/VNet/data/val/t2/003_rh.nii.gz')
img_test_data = img_test.get_fdata()
img_test_data = img_test_data.reshape((1,) + img_test_data.shape + (1,))

In [ ]:
img_test_data.shape

(1, 40, 40, 40, 1)

In [ ]:
prediction = model.predict(img_test_data)

In [ ]:
predict_nii = nib.Nifti1Image(prediction[0][:,:,:,0], img_test.affine, img_test.header)
nib.save(predict_nii, './drive/MyDrive/VNet/data/003_rh_prediction.nii.gz')

In [ ]:
prediction.shape

(1, 40, 40, 40, 1)